In [ ]:
import numpy as np
import panel as pn
import ipywidgets as ipw

pn.extension('ipywidgets')

The `IPyWidget` pane renders most [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/) (also known as *Jupyter widgets*) both in the notebook and in a deployed server. This allows leveraging the growing ecosystem directly within Panel by simply wrapping the component in the `IPyWidget` pane. For a list of `ipywidgets`, check out [best-of-jupyter](https://github.com/ml-tooling/best-of-jupyter#interactive-widgets--visualization).

Panel works especially well with `ipywidgets` built on top of [`AnyWidget`](https://anywidget.dev/en/getting-started/). See the [`AnyWidget` Community Page](https://anywidget.dev/en/community/) for a gallery of widgets you can use with Panel.

#### Prerequisites

To use `ipywidgets` with Panel in a server context, you must install the [`ipywidgets_bokeh`](https://github.com/bokeh/ipywidgets_bokeh) package:

```bash
pip install ipywidgets_bokeh
```

and import the `ipywidgets` extension:

```python
pn.extension("ipywidgets")
```

In a notebook, this is not necessary since Panel uses the regular notebook ipywidget renderer.

#### Parameters:

For details on other options for customizing the component, see the [layout](../../how_to/layout/index.md) and [styling](../../how_to/styling/index.md) how-to guides.

* **`object`** (object): The ipywidget object being displayed.

##### Display

* **`default_layout`** (pn.layout.Panel, default=Row): Layout to wrap the plot and widgets in.

___

The `IPyWidget` pane will automatically display the `ipywidget` object while keeping all its interactive features:

In [ ]:
date   = ipw.DatePicker(description='Date')
slider = ipw.FloatSlider(description='Float')
play   = ipw.Play()

layout = ipw.HBox(children=[date, slider, play])

ipywidget_pane = pn.pane.IPyWidget(layout)
ipywidget_pane

You can use `pn.panel` as an alternative to `pn.pane.IPyWidget`:

In [ ]:
pn.panel(layout)

## Updates

You can update the `.object` of the `IPyWidget` pane:

In [ ]:
import ipyleaflet as ipyl

cities = {
    "London": (51.5074, 0.1278),
    "Paris": (48.8566, 2.3522),
    "New York": (40.7128, -74.0060)
}

city = pn.widgets.Select(name="City", options=list(cities))
container = pn.pane.IPyWidget(width=500)

def update_container(city, container=container):
    container.object = ipyl.Map(zoom=4, center=cities[city])
city.rx.watch(update_container)
update_container(city.value)
    
pn.Column(city, container)

## Efficient Updates

In the previous section, you may have noticed the screen *flicker* when selecting another city in the dropdown. This happens because we create and display the map from scratch on user interactions.

To avoid screen *flicker* and update more efficiently, **you should update the existing `ipywidget` *in-place* whenever possible**:

In [ ]:
import ipyleaflet as ipyl

cities = {
    "London": (51.5074, 0.1278),
    "Paris": (48.8566, 2.3522),
    "New York": (40.7128, -74.0060)
}

city = pn.widgets.Select(name="City", options=list(cities))
leaflet_map = ipyl.Map(zoom=4, center=cities[city.value])
container = pn.pane.IPyWidget(leaflet_map, width=500)

def update_container(city, leaflet_map=leaflet_map):
    leaflet_map.center = cities[city]
city.rx.watch(update_container)
    
pn.Column(city, container)

Try selecting another city and watch the map update with a nice, smooth transition.

### Respond to User Input

You may respond to user input using `bind`, `sync_with_widget`, `create_rx`, event callbacks, the [traitlets observer pattern](https://traitlets.readthedocs.io/en/stable/using_traitlets.html#observe).

We highly recommend checking out the how-to guide [Observing ipywidgets](../../how_to/observers/ipywidget.md) for much more functionality, inspiration and detail.

### `pn.bind`

Any ipywidget with a `value` parameter can be used with `pn.bind`. For example, here we declare a function that binds to the value of a `FloatSlider`:

In [ ]:
slider = ipw.IntSlider(description='Slider', min=-5, max=5)

def cb(value):
    return 'The slider value is ' + (
        'negative' if value < 0 else 'nonnegative'
    )

pn.Row(slider, pn.bind(cb, slider))

### `sync_with_widget`

You can synchronize an ipywidget with a Panel widget using the `sync_with_widget` method:

In [ ]:
import ipyleaflet as ipyl

cities = {
    "London": (51.5074, 0.1278),
    "Paris": (48.8566, 2.3522),
    "New York": (40.7128, -74.0060)
}

city = pn.widgets.Select(name="City", options=list(cities))
leaflet_map = ipyl.Map(zoom=4, center=cities[city.value])

zoom_widget = pn.widgets.FloatSlider(value=2.0, start=1.0, end=24.0, name="Zoom")
zoom_control_widget = pn.widgets.Checkbox(value=True, name="Show Zoom Control")

pn.observers.ipywidget.sync_with_widget(leaflet_map, zoom_widget, "zoom")
pn.observers.ipywidget.sync_with_widget(leaflet_map, zoom_control_widget, "zoom_control")
pn.Column(leaflet_map, zoom_widget, zoom_control_widget)

This example only scratches the surface of what's available in the `panel.observers.ipywidget` namespace. **Check out out the how-to guide [Observing ipywidgets](../../how_to/observers/ipywidget.md) for much more functionality, inspiration and detail**.

### `create_rx`

You can create reactive values that are bidirectionally synchronized to an ipywidget using the `create_rx` method.

In [ ]:
import ipyleaflet as ipyl
import panel as pn

pn.extension()

cities = {
    "London": (51.5074, 0.1278),
    "Paris": (48.8566, 2.3522),
    "New York": (40.7128, -74.0060)
}

city = pn.widgets.Select(name="City", options=list(cities))
leaflet_map = ipyl.Map(zoom=4, center=cities[city.value])

zoom, zoom_control = pn.observers.ipywidget.create_rx(leaflet_map, "zoom", "zoom_control")

pn.Column(leaflet_map, zoom.rx.pipe(lambda x: f"**Value**: {x}, **Zoom Control**: " + zoom_control.rx.pipe(str)))

In [ ]:
zoom.rx.value=2

Lets change back the zoom

In [ ]:
zoom.rx.value=4

For more detail see the how-to guide [Wrapping ipywidgets](../../how_to/observers/ipywidget.md).

### Event Callbacks

Sometimes, you may want to capture user interaction that isn’t available through a widget trait. For example, `ipyleaflet.CircleMarker` has an `.on_click()` method that allows you to execute a callback when a marker is clicked. In this case, you may want to define a callback that handles the event.

In [ ]:
import ipyleaflet as ipyl

import panel as pn

pn.extension("ipywidgets")

london = (51.5, 359.9)

# Stores the number of clicks
n_clicks = pn.rx(0)


# A click callback that updates the reactive value
def on_click(**kwargs):
    n_clicks.rx.value += 1


# Create the map, add the CircleMarker, and register the map with Shiny
marker = ipyl.CircleMarker(location=london)
marker.on_click(on_click)
map_ = ipyl.Map(center=london, zoom=7)
map_.add_layer(marker)

clicks_text = pn.rx("**Number of clicks: {clicks}**").format(clicks=n_clicks)
pn.Column(map_, pn.pane.Markdown(clicks_text))

Click the blue circle and observe how the text below the map updates.

To read more about event callbacks, see the [Widget Events](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Events.html) section of the ipywidgets documentation.

### `traitlets` `observe` method

If you are already familiar with the [traitlets observer pattern](https://traitlets.readthedocs.io/en/stable/using_traitlets.html#observe) you may also use this API.

In [ ]:
caption = ipw.Label(value='The slider value is nonnegative')
slider = ipw.IntSlider(min=-5, max=5, value=1, description='Slider')

def handle_slider_change(change):
    caption.value = 'The slider value is ' + (
        'negative' if change.new < 0 else 'nonnegative'
    )

slider.observe(handle_slider_change, names='value')

pn.Row(slider, caption)

## Sizing

You might need to experiment with the the ipywidget `.layout.height` and `.layout.width` parameters and the `IPyWidget` pane `.height`, `width`, and `sizing_mode` parameters to get our use case correctly sized.

Let's start with a simple example to illustrate the challenge: Nothing is visible in the first output cell below.

In [ ]:
import ipyleaflet as ipyl
import panel as pn

pn.extension("ipywidgets")

map = ipyl.Map(zoom=4)
pn.pane.IPyWidget(map, height=200)

Let's add a border to see what happens.

In [ ]:
map = ipyl.Map(zoom=4)
pn.pane.IPyWidget(map, height=200, styles={"border": "1px solid black"})

We can see that the width is close to zero.

We can solve this problem by setting the `ipywidgets` width and height to `"100%"` and then controlling the `height`, `width` and/ or `sizing_mode` of the `IPyWidget` pane.

In [ ]:
map = ipyl.Map(zoom=4)
map.layout.width = map.layout.height = "100%"
pn.pane.IPyWidget(map, height=200, width=200, styles={"border": "1px solid black"})

In [ ]:
map = ipyl.Map(zoom=4)
map.layout.width = map.layout.height = "100%"
pn.pane.IPyWidget(map, height=300, sizing_mode="stretch_width", styles={"border": "1px solid black"})

## Example: LonBoard

For a larger example, check out how Panel works with [`lonboard`](https://developmentseed.org/lonboard/latest/) [here](https://developmentseed.org/lonboard/latest/ecosystem/panel/).

[![Panel lonboard example](https://assets.holoviz.org/panel/examples/panel-lonboard-application.gif)](https://developmentseed.org/lonboard/latest/ecosystem/panel/)

## More Examples

You can find specific examples by searching our [Discourse](https://discourse.holoviz.org/) site. Remember to share your examples too. Thanks.

## Limitations

The ipywidgets support has some limitations because it integrates two very distinct ecosystems. In particular, it is not yet possible to set up JS-linking between a Panel and an ipywidget object or support embedding. These limitations are not fundamental technical limitations and may be solved in the future.

## For Developers

If you want to integrate your ipywidget better with Panel, we highly recommend using the [`AnyWidget`](https://anywidget.dev/) framework to develop that ipywidget.

If you want to convert your `ipywidget` to a Panel native widget, you can do so with Panel's [`AnyWidgetComponent`](../custom_components/AnyWidgetComponent.ipynb), [`JSComponent`](../custom_components/JSComponent.ipynb), or [`ReactComponent`](../custom_components/ReactComponent.ipynb).